In [1]:
import cv2
import glob
import numpy as np
import math

In [2]:
def find_circles(thresh_frame):
    contours, hierarchy = cv2.findContours(thresh_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#
    contours_cirles = []
    for con in contours:
        perimeter = cv2.arcLength(con, True)
        area = cv2.contourArea(con)
        if perimeter == 0 or area>500:
            return []
        circularity = 4*math.pi*(area/(perimeter*perimeter))
        
        if 0.8 < circularity < 1.2:
            contours_cirles.append(con)
    return contours_cirles

# Using Red color channels from RGB (issue when there is lots of white spots where intensity for all RGB values are equally high)

In [3]:
cap = cv2.VideoCapture(r"../data/stable_background.mp4")
while True:
    ret, stable_frame = cap.read()           
    if not ret:
        break
    stable_frame_blurred = cv2.GaussianBlur(stable_frame,(11,11),0)
    img_r = stable_frame_blurred[:,:,0]
    threshold = int(img_r.max()*0.95)
    ret,thresh_frame = cv2.threshold(img_r,threshold,255,cv2.THRESH_BINARY)
    circles = find_circles(thresh_frame)
    for c in circles:
        M = cv2.moments(c)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv2.drawMarker(stable_frame, (cx, cy), color=[0, 255, 0], thickness=2, markerType= cv2.MARKER_TILTED_CROSS, line_type=cv2.LINE_AA,markerSize=7)



    #cv2.drawContours(stable_frame, circles, 0, (0,255,0), 3)
    cv2.imshow("frame", stable_frame)
    cv2.imshow("filtered", thresh_frame)
    if cv2.waitKey(10) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        print ("Exiting program")
cv2.destroyAllWindows()

# Using LAB color spaces (A channel)

In [4]:
cap = cv2.VideoCapture(r"../data/laser-treatment.mp4")
while True:
    ret, stable_frame = cap.read()           
    if not ret:
        break
    lab = cv2.cvtColor(stable_frame, cv2.COLOR_RGB2LAB)
    L,A,B=cv2.split(lab)
    stable_frame_blurred = cv2.GaussianBlur(A,(15,15),0)
    threshold = int(stable_frame_blurred.max()*0.95)
    #print(f"thresh: {threshold}")
    ret,thresh_frame = cv2.threshold(stable_frame_blurred,threshold,255,cv2.THRESH_BINARY)
    circles = find_circles(thresh_frame)
    for c in circles:
        M = cv2.moments(c)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv2.drawMarker(stable_frame, (cx, cy), color=[0, 255, 0], thickness=2, markerType= cv2.MARKER_TILTED_CROSS, line_type=cv2.LINE_AA,markerSize=7)

    #cv2.drawContours(stable_frame, circles, 0, (0,255,0), 3)
    cv2.imshow("stable_frame", stable_frame)
    cv2.imshow("thresh_frame", thresh_frame)
    cv2.imshow("A", A)
    if cv2.waitKey(10) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        print ("Exiting program")
cv2.destroyAllWindows()

# Using HSV to filter red

In [5]:
cap = cv2.VideoCapture(r"../data/unstable_background.mp4")
while True:
    ret, frame = cap.read()           
    if not ret:
        break
    
    
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    frame_blurred = cv2.GaussianBlur(frame,(5,5),0)

    # Define lower and upper HSV boundaries for the color blue
    lower_red_1 = np.array([0, 50, 150])
    upper_red_1 = np.array([10, 255, 255])


    # Create a mask with cv2.inRange to detect red colors
    mask = cv2.inRange(frame_blurred, lower_red_1, upper_red_1)

    
    # Use bitwise AND to extract the red color from the original image

    circles = find_circles(mask)
    cv2.drawContours(frame, circles, 0, (0,255,0), 3)
    
    cv2.imshow("mask", mask)
    cv2.imshow("original", frame)
    if cv2.waitKey(10) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        print ("Exiting program")
cv2.destroyAllWindows()

Exiting program


In [6]:
cv2.destroyAllWindows()

# LAB color space seems to have worked best for red colored lasers